In [0]:

# Replace with your Event Hub connection string
connectionString = "Endpoint=sb://traffic-demo.servicebus.windows.net/;SharedAccessKeyName=traffic-policy;SharedAccessKey=PU8yH9p9XVMvnU5odDnUWCxddYrGUbkYu+AEhJpOVMI=;EntityPath=traffic-iot"

ehConf = {
  'eventhubs.connectionString' : connectionString
}

# Read stream from Event Hub
df = spark.readStream \
    .format("eventhubs") \
    .options(**ehConf) \
    .load()

df.printSchema()


root
 |-- body: binary (nullable = true)
 |-- partition: string (nullable = true)
 |-- offset: string (nullable = true)
 |-- sequenceNumber: long (nullable = true)
 |-- enqueuedTime: timestamp (nullable = true)
 |-- publisher: string (nullable = true)
 |-- partitionKey: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- systemProperties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [0]:
%pip install azure-eventhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/73.1 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 71.7/73.1 kB 4.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/327.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 327.7/327.8 kB 37.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 6.2 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from azure.eventhub import EventHubConsumerClient

CONNECTION_STR = "Endpoint=sb://traffic-demo.servicebus.windows.net/;SharedAccessKeyName=traffic-policy;SharedAccessKey=PU8yH9p9XVMvnU5odDnUWCxddYrGUbkYu+AEhJpOVMI=;EntityPath=traffic-iot"
CONSUMER_GROUP = "traffic-app"  # or $Default consumer group
EVENTHUB_NAME = "traffic-iot"

def on_event(partition_context, event):
    print(f"Received event from partition: {partition_context.partition_id}")
    print(event.body_as_str())
    partition_context.update_checkpoint(event)

client = EventHubConsumerClient.from_connection_string(
    conn_str=CONNECTION_STR,
    consumer_group=CONSUMER_GROUP,
    eventhub_name=EVENTHUB_NAME
)

with client:
    client.receive(
        on_event=on_event,
        starting_position="-1",  # "-1" = read from beginning of stream
    )


Received event from partition: 0
{"vehicle_id": 9332, "speed": 110, "timestamp": "2025-08-28 18:43:13.731219"}
Received event from partition: 0
{"vehicle_id": 7079, "speed": 117, "timestamp": "2025-08-28 18:43:18.922162"}
Received event from partition: 0
{"vehicle_id": 4978, "speed": 72, "timestamp": "2025-08-28 18:43:24.261847"}
Received event from partition: 1
{"vehicle_id": 4925, "speed": 79, "timestamp": "2025-08-28 18:43:27.841534"}
Received event from partition: 0
{"vehicle_id": 8024, "speed": 113, "timestamp": "2025-08-28 18:43:35.548794"}
Received event from partition: 1
{"vehicle_id": 6605, "speed": 20, "timestamp": "2025-08-28 18:43:31.710027"}
Received event from partition: 0
{"vehicle_id": 8417, "speed": 114, "timestamp": "2025-08-28 18:43:39.851699"}
Received event from partition: 1
{"vehicle_id": 2450, "speed": 68, "timestamp": "2025-08-28 18:43:47.834024"}
Received event from partition: 0
{"vehicle_id": 1889, "speed": 120, "timestamp": "2025-08-28 18:43:43.913644"}
Recei

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
from pyspark.sql.types import StructType, StringType
from pyspark.sql.functions import col, from_json, coalesce, to_timestamp

# ---- 1) Schema: keep everything as STRING, cast later (avoids nulls on type mismatch)
schema = (StructType()
          .add("vehicle_id", StringType())
          .add("speed", StringType())
          .add("timestamp", StringType())     # some producers use "timestamp"
          .add("event_time", StringType()))   # others use "event_time"

# ---- 2) Secure your connection string (example uses a placeholder)
# connectionString = dbutils.secrets.get("scope", "eh-conn")   # recommended
connectionString ="Endpoint=sb://traffic-demo.servicebus.windows.net/;SharedAccessKeyName=traffic-policy;SharedAccessKey=PU8yH9p9XVMvnU5odDnUWCxddYrGUbkYu+AEhJpOVMI=;EntityPath=traffic-iot"

eh_conf = {
    "eventhubs.connectionString": sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString),
    "eventhubs.consumerGroup": "$Default",
    # Start from the earliest retained events
    "eventhubs.startingPosition": """{
      "offset": "-1",
      "seqNo": -1,
      "enqueuedTime": null,
      "isInclusive": true
    }"""
}

# ---- 3) Read Event Hubs stream
raw_stream = (spark.readStream
              .format("eventhubs")
              .options(**eh_conf)
              .load())

# ---- 4) Parse JSON safely
parsed = raw_stream.select(
    col("enqueuedTime").alias("ingest_time"),
    from_json(col("body").cast("string"), schema).alias("data")
)

iot_df = (parsed
    .select(
        col("data.vehicle_id").cast("int").alias("vehicle_id"),
        col("data.speed").cast("int").alias("speed"),
        # prefer payload time; fall back to Event Hubs enqueued time
        coalesce(col("data.timestamp"), col("data.event_time")).alias("ts_raw"),
        col("ingest_time")
    )
    .withColumn(
        "event_time",
        coalesce(
            to_timestamp(col("ts_raw")),                     # try default parsing
            col("ingest_time").cast("timestamp")             # fallback
        )
    )
    .drop("ts_raw")
)

# ---- 5) Write to Delta (BRONZE) with a dedicated checkpoint
delta_query = (iot_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", "/mnt/bronze/_checkpoint_iot")
    .option("mergeSchema", "true")    # allow new/changed columns
    .start("/mnt/bronze/iotdata"))

# ---- 6) OPTIONAL: Also stream to console so you can verify rows immediately
debug_query = (iot_df.writeStream
    .format("console")
    .option("truncate", False)
    .option("numRows", 20)
    .start())


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json, col, to_timestamp, current_timestamp

# 1) Schema
schema = StructType([
    StructField("vehicle_id", IntegerType(), True),
    StructField("speed", IntegerType(), True),
    StructField("timestamp", StringType(), True)  # raw string
])

# 2) Parse + Add event_time
parsed_df = (raw_stream
    .select(from_json(col("body").cast("string"), schema).alias("data"))
    .select("data.*")
    .withColumn("event_time", to_timestamp("timestamp"))  # convert to timestamp
    .withColumn("ingest_time", current_timestamp())       # optional lineage
)

# 3) Paths
silver_path = "dbfs:/mnt/silver/iotdata"
checkpoint_path = "dbfs:/mnt/silver/_checkpoint_iot"

# 4) Write to Silver (Delta Lake)
delta_query = (parsed_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)
    .option("mergeSchema", "true")   # allow schema evolution
    .start(silver_path)
)

# 5) Debug Console Sink (for monitoring)
console_query = (parsed_df.writeStream
    .format("console")
    .outputMode("append")
    .option("truncate", False)
    .start()
)


In [0]:

# 6) ---- In a separate cell, AFTER stream runs ----
# Read from Silver Delta table
silver_df = spark.read.format("delta").load(silver_path)

# Show only 10 rows
silver_df.show(10, truncate=False)


+----------+-----+--------------------------+--------------------------+-----------------------+
|vehicle_id|speed|timestamp                 |event_time                |ingest_time            |
+----------+-----+--------------------------+--------------------------+-----------------------+
|4925      |79   |2025-08-28 18:43:27.841534|2025-08-28 18:43:27.841534|2025-08-28 13:29:39.478|
|6605      |20   |2025-08-28 18:43:31.710027|2025-08-28 18:43:31.710027|2025-08-28 13:29:39.478|
|2450      |68   |2025-08-28 18:43:47.834024|2025-08-28 18:43:47.834024|2025-08-28 13:29:39.478|
|9278      |52   |2025-08-28 18:43:56.346297|2025-08-28 18:43:56.346297|2025-08-28 13:29:39.478|
|9002      |112  |2025-08-28 18:44:00.838685|2025-08-28 18:44:00.838685|2025-08-28 13:29:39.478|
|8900      |44   |2025-08-28 18:44:04.837131|2025-08-28 18:44:04.837131|2025-08-28 13:29:39.478|
|1245      |99   |2025-08-28 18:44:16.873968|2025-08-28 18:44:16.873968|2025-08-28 13:29:39.478|
|1877      |83   |2025-08-28 1

In [0]:
from pyspark.sql.functions import col, avg, count, window

# 1. Read from silver (streaming)
silver_df = (
    spark.readStream
    .format("delta")
    .load("/mnt/silver/iotdata")
)

# 2. Aggregate for gold
gold_df = (
    silver_df
    .withWatermark("event_time", "10 minutes")   # use watermark on event_time
    .groupBy(
        window(col("event_time"), "5 minutes"),  # tumbling window of 5 mins
        col("vehicle_id")
    )
    .agg(
        avg("speed").alias("avg_speed"),
        count("*").alias("event_count")
    )
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("vehicle_id"),
        col("avg_speed"),
        col("event_count")
    )
)

# 3. Write to gold (Delta table)
query = (
    gold_df.writeStream
    .format("delta")
    .outputMode("append")     # required for aggregates with watermark
    .option("checkpointLocation", "/mnt/gold/_checkpoint_iot")
    .start("/mnt/gold/iotdata")
)


In [0]:
# Register the gold delta folder as a temporary view
gold_df_read = spark.read.format("delta").load("/mnt/gold/iotdata")

# Show 10 rows
gold_df_read.show(10, truncate=False)

+-------------------+-------------------+----------+---------+-----------+
|window_start       |window_end         |vehicle_id|avg_speed|event_count|
+-------------------+-------------------+----------+---------+-----------+
|2025-08-28 18:40:00|2025-08-28 18:45:00|8322      |87.0     |1          |
|2025-08-28 18:40:00|2025-08-28 18:45:00|2047      |60.0     |1          |
|2025-08-28 18:40:00|2025-08-28 18:45:00|7314      |55.0     |1          |
|2025-08-28 18:40:00|2025-08-28 18:45:00|6605      |20.0     |1          |
|2025-08-28 18:40:00|2025-08-28 18:45:00|7634      |71.0     |1          |
|2025-08-28 18:40:00|2025-08-28 18:45:00|4879      |20.0     |1          |
|2025-08-28 18:40:00|2025-08-28 18:45:00|9278      |52.0     |1          |
|2025-08-28 18:40:00|2025-08-28 18:45:00|8417      |114.0    |1          |
|2025-08-28 18:40:00|2025-08-28 18:45:00|4925      |79.0     |1          |
|2025-08-28 18:40:00|2025-08-28 18:45:00|1245      |99.0     |1          |
+-------------------+----

In [0]:
%sql
SELECT window_start, vehicle_id, avg_speed, event_count
FROM delta.`/mnt/gold/iotdata`
ORDER BY window_start DESC
LIMIT 10;


window_start,vehicle_id,avg_speed,event_count
2025-08-28T18:45:00Z,7030,93.0,1
2025-08-28T18:45:00Z,2467,91.0,1
2025-08-28T18:45:00Z,8615,97.0,1
2025-08-28T18:45:00Z,7130,101.0,1
2025-08-28T18:45:00Z,2860,66.0,1
2025-08-28T18:45:00Z,4514,41.0,1
2025-08-28T18:45:00Z,2309,93.0,1
2025-08-28T18:45:00Z,1878,40.0,1
2025-08-28T18:45:00Z,5015,116.0,1
2025-08-28T18:45:00Z,8787,103.0,1
